# CFM 101 Final Project

### Project Category: Market-Meet

### Group Member: Max Sun, Sean Lee, Rain Luo

#### Intial strategy:
In terms of overall strategy, we sought to replicate the market, matching the S&P 500 and TSX 60 sector composition. The simulated portfolio remains diversified by weighting stocks according to the sector’s actual market representation. For each sector, the algorithm identifies the stocks that have the highest correlation with the sector’s performance, selecting those stocks that move most similarly to the industry as they represent its behavior best. Once these stocks are identified, the strategy allocates more weight to the stocks that are most correlated with the sector, with the number of stocks selected based on the sector’s weight in the index. Specifically, for each industry, the top N stocks are chosen, where N is calculated as a percentage of the sector’s weight in the index, multiplied by 25. Additionally, the allocation progressively decreases for each subsequent stock in the industry. The strategy aims to backtest this portfolio construction to simulate potential profits by using historical data, allowing it to assess whether this sector-based selection method can outperform broader market indices. Essentially, the strategy seeks to leverage sector-specific performance to build a portfolio that could potentially beat the market.

## Design decisions 
1. We discovererd that for one particular stock, it can have a a high corrlation to mutiple sectors of the S&P. We have decided to put a maximium cap.
2. We decided to use a year worth of past trend. From discussing with other groups 

### Set up
1. We found out that there are tickers on yfinance that trackes the S&P by sectors, they are:

2. We also included the sectors composition in the S&P 500 and TSX 60, which we will use later on.

| Sector                 | Composition in S&P500 (%) | Composition in TSX60 (%) | S&P Ticker | TSX Ticker  |
|------------------------|----------------------|--------------------|-----------------|---------------|
| Basic Materials        | 1.71              | 8.49            | ^SP500-15       | ^GSPTTMT      |
| Industrials            | 7.19              | 13.11            | ^SP500-20       | ^GSPTTIN      |
| Consumer Cyclical      | 10.75              | 5.31            | ^SP500-25       | ^GSPTTCD      |
| Consumer Defensive     | 5.76              | 5.09            | ^SP500-30       | ^GSPTTCS      |
| Healthcare             | 10.14              | 0            | ^SP500-35       | ^GSPTTHC      |
| Financial Services     | 13.03              | 33.87            | ^SP500-40       | ^SPTTFS       |
| Technology             | 30.45              | 9.63            | ^SP500-45       | ^SPTTTK       |
| Communication Services | 13.40              | 3.04            | ^SP500-50       | ^GSPTTTS      |
| Utilities              | 2.35              | 3.18            | ^SP500-55       | ^GSPTTUT      |
| Real Estate            | 2.07              | 0.62            | ^SP500-60       | ^GSPTTRE      |
| Energy                 | 3.15              | 17.66            | ^SP500-1010     | ^SPTTEN       |


In [37]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import threading
import sys

attributes = ['sector', 'exchange', 'currency', 'marketCap']
start_date = '2023-10-01'
end_date = '2024-10-01'


# Each industry is mapped to (% share of S&P500, % share of TSX60, S&P industry ticker, TSX60 capped industry ticker)
# To obtain values for % share, run market_by_sector('SP500') and market_by_sector('TSX60'), respectively (see next cell)
# Since % share changes quarterly, we don't need to run this every time
sectors = {
    'Basic Materials': (0.0171, 0.0849, '^SP500-15', '^GSPTTMT'),
    'Industrials': (0.0719, 0.1311, '^SP500-20', '^GSPTTIN'),
    'Consumer Cyclical': (0.1075, 0.0531, '^SP500-25', '^GSPTTCD'),
    'Consumer Defensive': (0.0576, 0.0509, '^SP500-30', '^GSPTTCS'),
    'Healthcare': (0.1014, 0.0000, '^SP500-35', '^GSPTTHC'),
    'Financial Services': (0.1303, 0.3387, '^SP500-40', '^SPTTFS'),
    'Technology': (0.3045, 0.0963, '^SP500-45', '^SPTTTK'),
    'Communication Services': (0.1340, 0.0304, '^SP500-50', '^GSPTTTS'),
    'Utilities': (0.0235, 0.0318, '^SP500-55', '^GSPTTUT'),
    'Real Estate': (0.0207, 0.0062, '^SP500-60', '^GSPTTRE'),
    'Energy': (0.0315, 0.1766, '^SP500-1010', '^SPTTEN')
}

### Fetch Data

Here is where we organized the data. We decided to filter out stocks that have less than 100,000 daily volume. 

In [38]:
# adds ticker info to data df
def get_data(ticker, data, history, filter):
    yf_data = yf.Ticker(ticker).info
    if(not filter):
        for att in attributes:
            if(att not in yf_data):
                print(ticker, 'missing', att)
                continue
            data.loc[ticker, att] = yf_data[att]
        hist = yf.Ticker(ticker).history(start=start_date, end=end_date)
        history[ticker] = hist['Close'].pct_change().dropna()
        return
    # check if stock is CAD or USD
    if('currency' not in yf_data or yf_data['currency'] not in ['USD', 'CAD']):
        data.drop(ticker, inplace=True)
        print('Dropped', ticker)
        return
    for att in attributes:
        if(att not in yf_data):
            print(ticker, 'missing', att)
            continue
        data.loc[ticker, att] = yf_data[att]
    hist = yf.Ticker(ticker).history(start=start_date, end=end_date)
    history[ticker] = hist['Close'].pct_change().dropna()
    volume = hist['Volume'].resample('MS').sum()
    # Take all months with >= 18 trading days for volume calculation
    volume.drop([month for month in volume.index if hist.resample('MS').size().loc[month] < 18], inplace=True)
    # check if stock has at least 100,000 average monthly volume
    if(volume.mean() < 1e5):
        data.drop(ticker, inplace=True)

# returns df containing all ticker info
def get_tickers(file_name='Tickers.csv', filter=True):
    with threading.Lock():
        tickers = pd.read_csv(file_name, header=None)
        data = pd.DataFrame(index=[ticker for ticker in tickers[0]], columns=attributes)
        history = {}
        threads = [threading.Thread(target=get_data, args=(ticker,data,history,filter)) for ticker in tickers[0]]
        for thread in threads:
            thread.start()
        for thread in threads:
            thread.join()
    return (data, history)

In [39]:
def weighted_max_bipartite_matching(N, M, A, C):
    """
    Finds the weighted maximum bipartite matching for sectors and stocks.
    
    Args:
    - N: Number of sectors.
    - M: Number of stocks.
    - A: List of length N where A[i] is the number of stocks required by sector i.
    - C: 2D list (N x M) of weights (correlations) between sectors and stocks.
    
    Returns:
    - match: List of tuples (sector, stock) representing the matching.
    - total_weight: Total weight of the matching.
    """
    # Expand the graph: create dummy nodes for each sector demand
    total_sectors = sum(A)
    expanded_C = [[-sys.maxsize] * M for _ in range(total_sectors)]
    
    sector_mapping = []
    index = 0
    for i in range(N):
        for _ in range(A[i]):
            expanded_C[index] = C[i]
            sector_mapping.append(i)  # Map expanded sector to original sector
            index += 1

    # Hungarian algorithm for max-weight matching
    match = [-1] * M  # Stores which sector is assigned to each stock
    sector_label = [0] * total_sectors
    stock_label = [0] * M
    slack = [0] * M
    slack_x = [-1] * M
    parent = [-1] * M
     
    def dfs(x, visited_x, visited_y):
        visited_x[x] = True
        for y in range(M):
            if visited_y[y]:
                continue
            delta = sector_label[x] + stock_label[y] - expanded_C[x][y]
            if delta == 0:  # Tight edge
                visited_y[y] = True
                if match[y] == -1 or dfs(match[y], visited_x, visited_y):
                    match[y] = x
                    return True
            else:  # Update slack
                if slack[y] > delta:
                    slack[y] = delta
                    slack_x[y] = x
        return False

    # Initialize labels
    for x in range(total_sectors):
        sector_label[x] = max(expanded_C[x])

    # Augmenting path search
    for x in range(total_sectors):
        slack = [sys.maxsize] * M
        slack_x = [-1] * M
        while True:
            visited_x = [False] * total_sectors
            visited_y = [False] * M
            if dfs(x, visited_x, visited_y):
                break
            # Update labels
            delta = min(slack[y] for y in range(M) if not visited_y[y])
            for i in range(total_sectors):
                if visited_x[i]:
                    sector_label[i] -= delta
            for y in range(M):
                if visited_y[y]:
                    stock_label[y] += delta
                else:
                    slack[y] -= delta

    # Extract results
    total_weight = 0
    final_match = []
    for y in range(M):
        if match[y] != -1:
            sector_idx = sector_mapping[match[y]]
            final_match.append((sector_idx, y))
            total_weight += C[sector_idx][y]

    return final_match

#### Performance & Correlation Analysis

Here, is the core of our algorithm. We decided to select stocks by their correlation to different sectors. First, we will be determining the correlation/ beta between the sectors and individual stock. From there, we will be selecting the stocks that have the most correlation with a specific sectors. In theory, a stock from technology can be selected to repersent the performance for other sectors other than tech. However, this is unlikely to happen, since the sectors indices only contains the stock of that specific sector. 

We have implemented functions to compute beta and correlation of each stock with each index. Throught testing, we decided to go with [insert method], as it does out-perform [insert other method]. This method provides a more direct measure of how closely the stock movements align with the sector's performance over time.

We start constructing the portfolio by calculating the correlation and beta for each stock relative to the sectors. These calculations identifies the stocks that will most accurately represent their respective sectors in the portfolio. Our goal is to maintain a balance between sectors while minimizing risk through diversification.

Next, we apply the max percentage rule to ensure that no single stock makes up more than 15% of the portfolio. Optimizing through binary search ensures we can find the optimal stock allocation, allowing us to include 24 stocks in the final portfolio while maintaining the diversification constraints. The create_portfolio function then selects stocks with the highest correlation to each sector, assigning each stock a weight based on the calculated correlations and the portfolio’s allocation rules. This maximum weight matching approach allows us to match stocks to sectors based on their correlation strength, making sure that stocks most aligned with a given sector are selected. This minimizes risk and diversifies the portfolio. 


In [46]:
# returns function of stocks to sectors as given by f
# 0 for S&P500, 1 for TSX60
def calc(data, history, f, index):
    sector_metric = {stock:{} for stock in data.index}
    for sector in sectors:
        for stock in data.index:
            if(sectors[sector][index] == 0):
                continue
            df = pd.DataFrame({stock: history[stock], sector: history[sectors[sector][2+index]]}).dropna()
            # calculate metric given a function f
            sector_metric[stock][sector] = f(df, stock, sector)
    return sector_metric

def beta(df, stock, sector):
    return df[stock].cov(df[sector])/df[sector].var()

def corr(df, stock, sector):
    return df[stock].corr(df[sector])

# returns df containing history for each sector in TSX60
# since historical data for individual TSX60 sectors is unavailable, we take the weighted average of all stocks in each sector
def tsx_sectors():
    data, history = get_tickers('TSX60.csv', False)
    sector_history = pd.DataFrame({sectors[sector][3]: 0 for sector in sectors}, index=history[data.index[0]].index)
    total_market_cap = {sectors[sector][3]: 0 for sector in sectors}
    for stock in history:
        total_market_cap[sectors[data['sector'].loc[stock]][3]] += data['marketCap'].loc[stock]
    for stock in history:
        sector = sectors[data['sector'].loc[stock]][3]
        sector_history[sector] += history[stock]*data['marketCap'].loc[stock]/total_market_cap[sector]
    return sector_history

# returns df containing history for each sector in S&P500
def sp_sectors():
    history = {sectors[sector][2]: yf.Ticker(sectors[sector][2]).history(start=start_date, end=end_date)['Close'].pct_change().dropna() for sector in sectors}
    return pd.DataFrame(history, index=list(history.values())[0].index)

# returns sector percent change since start date
def aggregate_pct_change(history, stock):
    result = pd.Series(index=history[stock].index)
    prev = 1
    for day in history[stock].index:
        result[day] = prev*(1+history[stock][day])
        prev = result[day]
    return result

# binary search for optimal max percentage of a single stock such that we can have 24 stocks in our portfolio
def max_percentage(min_pct, max_stocks):
    low = min_pct
    high = 1.0
    while(low < high):
        mid = (low+high)/2
        sum = 0
        for sector in sectors:
            sum += min(max(1, sectors[sector][0]/2//mid), sectors[sector][0]/2//min_pct) + min(max(1, sectors[sector][1]/2//mid), sectors[sector][1]/2//min_pct)
        if(sum > max_stocks):
            low = mid+0.0001
        else:
            high = mid
    return round(low, 4)

def ticker_to_sector(ticker):
    for sector in sectors:
        if(ticker == sectors[sector][2]):
            return sector, 0
        if(ticker == sectors[sector][3]):
            return sector, 1
    return None

def create_portfolio(sector_corr, min_pct, max_pct, pct_cap):
    portfolio = {stock: 0 for stock in data.index}
    SECTOR_IDX = {}
    IDX_SECTOR = {}
    STOCK_IDX = {}
    IDX_STOCK = {}
    idx = 0
    for sector in sectors:
        SECTOR_IDX[sectors[sector][2]] = idx
        IDX_SECTOR[idx] = sectors[sector][2]
        SECTOR_IDX[sectors[sector][3]] = idx+1
        IDX_SECTOR[idx+1] = sectors[sector][3]
        idx += 2
    idx = 0
    for stock in data.index:
        for i in range(int(pct_cap/max_pct)):
            STOCK_IDX[stock] = idx
            IDX_STOCK[idx] = stock
            idx += 1
    CORR = [[] for _ in SECTOR_IDX]
    NUM_STOCKS = [0 for _ in SECTOR_IDX]
    sum = 0
    for sector in sectors:
        for i in range(2):
            if(sectors[sector][i]/2 < min_pct): # also checks if sector has no percentage
                continue
            for j in range(len(IDX_STOCK)):
                CORR[SECTOR_IDX[sectors[sector][2+i]]].append(int((1-abs(sector_corr[i][IDX_STOCK[j]][sector]-1))*1000)+10000)
            NUM_STOCKS[SECTOR_IDX[sectors[sector][2+i]]] = int(min(max(1, sectors[sector][i]/2//max_pct), sectors[sector][i]/2//min_pct))
            sum += NUM_STOCKS[SECTOR_IDX[sectors[sector][2+i]]]
    print('Total number of stocks:', sum)
    result = weighted_max_bipartite_matching(len(SECTOR_IDX), len(IDX_STOCK), NUM_STOCKS, CORR)
    for pair in result:
        sector, idx = ticker_to_sector(IDX_SECTOR[pair[0]])
        portfolio[IDX_STOCK[pair[1]]] += max_pct if sectors[sector][idx]/2 >= max_pct else round(sectors[sector][idx]/2, 4)
        print(IDX_STOCK[pair[1]], '->', sector, idx, '->', portfolio[IDX_STOCK[pair[1]]])
    for sector in sectors:
        for i in range(2):
            remaining = round(sectors[sector][i]/2 - sectors[sector][i]/2//max_pct*max_pct, 4)
            if(remaining == 0 or (sectors[sector][i]/2 >= min_pct and sectors[sector][i]/2 < max_pct)):
                continue
            best_corr = sector_corr[i][IDX_STOCK[0]][sector]
            best_stock = IDX_STOCK[0]
            for j in range(len(IDX_STOCK)):
                if(portfolio[IDX_STOCK[j]] != 0 and sector_corr[i][IDX_STOCK[j]][sector] > best_corr):
                    best_corr = sector_corr[i][IDX_STOCK[j]][sector]
                    best_stock = IDX_STOCK[j]
            portfolio[best_stock] += remaining
            print(best_stock, '->', sector, i, '->', portfolio[best_stock])
    sum = 0
    for stock in portfolio:
        portfolio[stock] = round(portfolio[stock], 4)
        sum += portfolio[stock]
    print('Total portfolio value:', sum)
    return portfolio

PCT_CAP = 0.07
MAX_STOCKS = 24
MIN_PCT = round(1/(2*MAX_STOCKS), 4)
MAX_PCT = max_percentage(MIN_PCT, MAX_STOCKS)
data, history = get_tickers()
tsx_by_sector = tsx_sectors()
for sector in tsx_by_sector:
    history[sector] = tsx_by_sector[sector]
sp_by_sector = sp_sectors()
for sector in sp_by_sector:
    history[sector] = sp_by_sector[sector]
sector_corr = [calc(data, history, corr, 0), calc(data, history, corr, 1)]
create_portfolio(sector_corr, MIN_PCT, MAX_PCT, PCT_CAP)

Dropped AGN
Dropped CELG
Dropped RTN
Dropped MON
Total number of stocks: 24
AAPL -> Technology 0 -> 0.0326
AAPL -> Technology 0 -> 0.0652
ABBV -> Healthcare 0 -> 0.0326
AMZN -> Communication Services 0 -> 0.0326
AMZN -> Communication Services 0 -> 0.0652
BAC -> Energy 1 -> 0.0326
BAC -> Financial Services 0 -> 0.0652
BK -> Energy 1 -> 0.0326
BLK -> Industrials 0 -> 0.0326
CAT -> Basic Materials 1 -> 0.0326
KO -> Consumer Defensive 0 -> 0.0288
QCOM -> Technology 0 -> 0.0326
QCOM -> Technology 0 -> 0.0652
RY.TO -> Financial Services 1 -> 0.0326
RY.TO -> Financial Services 1 -> 0.0652
SHOP.TO -> Technology 1 -> 0.0326
T.TO -> Consumer Defensive 1 -> 0.0255
TD.TO -> Financial Services 1 -> 0.0326
TD.TO -> Consumer Cyclical 1 -> 0.059199999999999996
TXN -> Consumer Cyclical 0 -> 0.0326
UNP -> Industrials 1 -> 0.0326
UNP -> Industrials 1 -> 0.0652
USB -> Financial Services 1 -> 0.0326
USB -> Financial Services 1 -> 0.0652
BLK -> Basic Materials 0 -> 0.0412
CAT -> Basic Materials 1 -> 0.04249

{'AAPL': 0.0652,
 'ABBV': 0.0507,
 'ABT': 0,
 'ACN': 0,
 'AIG': 0,
 'AMZN': 0.0882,
 'AXP': 0,
 'BA': 0,
 'BAC': 0.1209,
 'BB.TO': 0,
 'BIIB': 0,
 'BK': 0.0326,
 'BLK': 0.058,
 'BMY': 0,
 'C': 0,
 'CAT': 0.0583,
 'CL': 0,
 'KO': 0.0406,
 'LLY': 0,
 'LMT': 0,
 'MO': 0,
 'MRK': 0,
 'PEP': 0,
 'PFE': 0,
 'PG': 0,
 'PM': 0,
 'PYPL': 0,
 'QCOM': 0.0871,
 'RY.TO': 0.0875,
 'SHOP.TO': 0.0482,
 'T.TO': 0.0407,
 'TD.TO': 0.0592,
 'TXN': 0.0326,
 'UNH': 0,
 'UNP': 0.0656,
 'UPS': 0,
 'USB': 0.0652}

In [41]:
# adds market cap of ticker to data table
def ticker_by_sector(ticker, data):
    yf_data = yf.Ticker(ticker).info
    data[yf_data['sector']] += yf_data['marketCap']

# prints percentage of index in each sector
def market_by_sector(index):
    tickers = pd.read_csv(index+'.csv', header=None)
    data = {sector: 0 for sector in sectors}
    threads = [threading.Thread(target=ticker_by_sector, args=(ticker,data)) for ticker in tickers[0]]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    sum = 0
    for sector in data:
        sum += data[sector]
    for sector in data:
        print(sector, 'accounts for', round(data[sector]/sum*100, 2), 'percent of', index)